In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

2023-03-10 13:57:07.721314: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-03-10 13:57:10.129984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 13:57:10.130437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 13:57:10.130617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgr

In [2]:
train = Dataset("ecom_dataset/0001/train.parquet")
valid = Dataset("ecom_dataset/0002/valid.parquet")

In [3]:
target = 'sess_pid_seq'

In [4]:
# a couple of starter hyperparams

d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
n_epoch = 5

In [5]:
def get_model():
    mlp_block = mm.MLPBlock(
                    [128,d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

    schema = TensorflowMetadata.from_proto_text_file(
        '../',
        file_name='rees46_schema_modified.pbtxt'
    ).to_merlin_schema()

    train.schema = schema

    input_block = mm.InputBlockV2(
        train.schema.select_by_name('sess_pid_seq'),    
        embeddings=mm.Embeddings(
            train.schema.select_by_name('sess_pid_seq'), 
            sequence_combiner=None,
            dim=d_model
            ),
    #     pre=mm.StochasticSwapNoise()
    )

    train.schema = train.schema.select_by_name('sess_pid_seq')

    xlnet_block = mm.XLNetBlock(d_model=d_model, n_head=n_head, n_layer=n_layer)

    dense_block = mm.SequentialBlock(
        input_block,
        mlp_block,
        xlnet_block
    )

    mlp_block2 = mm.MLPBlock(
                    [128,d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    prediction_task = mm.CategoricalOutput(
        to_call=input_block["categorical"][target],
    )

    model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
                  metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[10])
                 )
    return model_transformer, xlnet_block

# Run 1

In [6]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/5


2023-03-10 13:57:17.631317: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 82s 110ms/step - loss: 8.8265 - recall_at_10: 0.0432 - mrr_at_10: 0.0166 - ndcg_at_10: 0.0228 - map_at_10: 0.0166 - precision_at_10: 0.0043 - regularization_loss: 0.0000e+00 - loss_batch: 8.8191
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 5.8014 - recall_at_10: 0.3091 - mrr_at_10: 0.1936 - ndcg_at_10: 0.2210 - map_at_10: 0.1936 - precision_at_10: 0.0309 - regularization_loss: 0.0000e+00 - loss_batch: 5.8019
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 4.1718 - recall_at_10: 0.5397 - mrr_at_10: 0.4080 - ndcg_at_10: 0.4394 - map_at_10: 0.4080 - precision_at_10: 0.0540 - regularization_loss: 0.0000e+00 - loss_batch: 4.1734
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.2806 - recall_at_10: 0.6585 - mrr_at_10: 0.5362 - ndcg_at_10: 0.5656 - map_at_10: 0.5362 - precision_at_10: 0.0658 - regularization_loss: 0.0000e+00 - loss_batch: 3.2849
Epoch 5/5
84/84 [=

{'loss': 8.810694694519043,
 'recall_at_10': 0.15318788588047028,
 'mrr_at_10': 0.06131112948060036,
 'ndcg_at_10': 0.08268804848194122,
 'map_at_10': 0.06131112948060036,
 'precision_at_10': 0.015318789519369602,
 'regularization_loss': 0.0,
 'loss_batch': 9.6568603515625}

# Run 2

In [7]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 9.1281 - recall_at_10: 0.0359 - mrr_at_10: 0.0128 - ndcg_at_10: 0.0181 - map_at_10: 0.0128 - precision_at_10: 0.0036 - regularization_loss: 0.0000e+00 - loss_batch: 9.1243
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.7038 - recall_at_10: 0.1907 - mrr_at_10: 0.1006 - ndcg_at_10: 0.1218 - map_at_10: 0.1006 - precision_at_10: 0.0191 - regularization_loss: 0.0000e+00 - loss_batch: 6.6971
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 4.9471 - recall_at_10: 0.4404 - mrr_at_10: 0.3077 - ndcg_at_10: 0.3393 - map_at_10: 0.3077 - precision_at_10: 0.0440 - regularization_loss: 0.0000e+00 - loss_batch: 4.9478
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.9842 - recall_at_10: 0.5607 - mrr_at_10: 0.4197 - ndcg_at_10: 0.4534 - map_at_10: 0.4197 - precision_at_10: 0.0561 - regularization_loss: 0.0000e+00 - loss_batch: 3.9878
Epoch 5/5
84/84 [=

{'loss': 8.971626281738281,
 'recall_at_10': 0.12817378342151642,
 'mrr_at_10': 0.05082216113805771,
 'ndcg_at_10': 0.06883765012025833,
 'map_at_10': 0.05082216113805771,
 'precision_at_10': 0.012817380018532276,
 'regularization_loss': 0.0,
 'loss_batch': 10.049013137817383}

# Run 3

In [8]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 8.9307 - recall_at_10: 0.0396 - mrr_at_10: 0.0142 - ndcg_at_10: 0.0201 - map_at_10: 0.0142 - precision_at_10: 0.0040 - regularization_loss: 0.0000e+00 - loss_batch: 8.9265
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 5.9376 - recall_at_10: 0.2951 - mrr_at_10: 0.1842 - ndcg_at_10: 0.2105 - map_at_10: 0.1842 - precision_at_10: 0.0295 - regularization_loss: 0.0000e+00 - loss_batch: 5.9350
Epoch 3/5
677/677 [==============================] - 75s 111ms/step - loss: 4.3616 - recall_at_10: 0.5184 - mrr_at_10: 0.3844 - ndcg_at_10: 0.4164 - map_at_10: 0.3844 - precision_at_10: 0.0518 - regularization_loss: 0.0000e+00 - loss_batch: 4.3657
Epoch 4/5
677/677 [==============================] - 75s 111ms/step - loss: 3.4916 - recall_at_10: 0.6319 - mrr_at_10: 0.5057 - ndcg_at_10: 0.5359 - map_at_10: 0.5057 - precision_at_10: 0.0632 - regularization_loss: 0.0000e+00 - loss_batch: 3.4969
Epoch 5/5
84/84 [=

{'loss': 8.798320770263672,
 'recall_at_10': 0.15647922456264496,
 'mrr_at_10': 0.05985381081700325,
 'ndcg_at_10': 0.08228185027837753,
 'map_at_10': 0.05985381081700325,
 'precision_at_10': 0.015647921711206436,
 'regularization_loss': 0.0,
 'loss_batch': 10.545936584472656}

# Run 4

In [9]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 8.8791 - recall_at_10: 0.0414 - mrr_at_10: 0.0155 - ndcg_at_10: 0.0215 - map_at_10: 0.0155 - precision_at_10: 0.0041 - regularization_loss: 0.0000e+00 - loss_batch: 8.8746
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 5.5817 - recall_at_10: 0.3289 - mrr_at_10: 0.2127 - ndcg_at_10: 0.2403 - map_at_10: 0.2127 - precision_at_10: 0.0329 - regularization_loss: 0.0000e+00 - loss_batch: 5.5795
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 3.8784 - recall_at_10: 0.5761 - mrr_at_10: 0.4489 - ndcg_at_10: 0.4793 - map_at_10: 0.4489 - precision_at_10: 0.0576 - regularization_loss: 0.0000e+00 - loss_batch: 3.8833
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.0679 - recall_at_10: 0.6797 - mrr_at_10: 0.5656 - ndcg_at_10: 0.5930 - map_at_10: 0.5656 - precision_at_10: 0.0680 - regularization_loss: 0.0000e+00 - loss_batch: 3.0749
Epoch 5/5
84/84 [=

{'loss': 8.63992691040039,
 'recall_at_10': 0.1588301658630371,
 'mrr_at_10': 0.06323756277561188,
 'ndcg_at_10': 0.0855293795466423,
 'map_at_10': 0.06323756277561188,
 'precision_at_10': 0.01588302105665207,
 'regularization_loss': 0.0,
 'loss_batch': 9.691500663757324}

# Run 5

In [10]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 8.9605 - recall_at_10: 0.0390 - mrr_at_10: 0.0141 - ndcg_at_10: 0.0199 - map_at_10: 0.0141 - precision_at_10: 0.0039 - regularization_loss: 0.0000e+00 - loss_batch: 8.9571
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.1194 - recall_at_10: 0.2618 - mrr_at_10: 0.1575 - ndcg_at_10: 0.1821 - map_at_10: 0.1575 - precision_at_10: 0.0262 - regularization_loss: 0.0000e+00 - loss_batch: 6.1199
Epoch 3/5
677/677 [==============================] - 75s 111ms/step - loss: 4.4762 - recall_at_10: 0.5000 - mrr_at_10: 0.3647 - ndcg_at_10: 0.3970 - map_at_10: 0.3647 - precision_at_10: 0.0500 - regularization_loss: 0.0000e+00 - loss_batch: 4.4783
Epoch 4/5
677/677 [==============================] - 75s 111ms/step - loss: 3.6222 - recall_at_10: 0.6166 - mrr_at_10: 0.4884 - ndcg_at_10: 0.5191 - map_at_10: 0.4884 - precision_at_10: 0.0617 - regularization_loss: 0.0000e+00 - loss_batch: 3.6248
Epoch 5/5
84/84 [=

{'loss': 8.899141311645508,
 'recall_at_10': 0.14763964712619781,
 'mrr_at_10': 0.05743885040283203,
 'ndcg_at_10': 0.07836496829986572,
 'map_at_10': 0.05743885040283203,
 'precision_at_10': 0.014763964340090752,
 'regularization_loss': 0.0,
 'loss_batch': 9.991716384887695}